# Simple prefix sum parallel (CUDA) GPU

Using the Blelloch algorithm.

## Single block simplified version

Some GPU + Windows + current CUDA.jl versions have a known bug with dynamic shared memory allocation.
So using static shared memory for running this computation under Windows 10.

This kernel only supports **power-of-two** n, for simplicity.

In [1]:
using CUDA

function kernel_blelloch_scan_static!(data, n)
    T = eltype(data)
    sdata = @cuStaticSharedMem(T, 1024)

    tid = threadIdx().x
    t = tid - 1  # 0-based

    if tid <= n
        sdata[tid] = data[tid]
    else
        return
    end
    sync_threads()

    # upsweep
    offset = 1
    while offset < n
        paircount = n ÷ (2 * offset)
        if t < paircount
            ai = offset*(2t+1)
            bi = offset*(2t+2)
            sdata[bi] += sdata[ai]
        end
        offset *= 2
        sync_threads()
    end

    if tid == 1
        sdata[n] = zero(T)
    end
    sync_threads()

    # downsweep
    offset = n ÷ 2
    while offset ≥ 1
        paircount = n ÷ (2 * offset)
        if t < paircount
            ai = offset*(2t+1)
            bi = offset*(2t+2)
            tmp = sdata[ai]
            sdata[ai] = sdata[bi]
            sdata[bi] += tmp
        end
        offset ÷= 2
        sync_threads()
    end

    data[tid] = sdata[tid]
    return
end

function gpu_scan_static!(x::CuArray)
    n = length(x)
    @assert ispow2(n)
    threads = n
    @cuda threads=threads kernel_blelloch_scan_static!(x, n)
end

gpu_scan_static! (generic function with 1 method)

In [2]:
N = 1024
x = CuArray(rand(Float32, N))
gpu_scan_static!(x)
collect(x)[1:10]

10-element Vector{Float32}:
 0.0
 0.5748641
 1.2661473
 2.2272089
 3.1423702
 3.785553
 4.3061175
 4.3568935
 4.7280483
 5.645898

CPU serial version for checking and comparison

In [3]:
using BenchmarkTools

# CPU reference exclusive scan (Blelloch-compatible)
function cpu_serial_scan(x::Vector{T}) where T
    n = length(x)
    out = Vector{T}(undef, n)
    out[1] = zero(T)
    for i in 2:n
        out[i] = out[i - 1] + x[i - 1]
    end
    return out
end

cpu_serial_scan (generic function with 1 method)

### Benchmarking

In [4]:
N = 1024           # must be power of 2, and max is 1024 bcz this is single block version
h_input = rand(Float32, N)

# CPU serial scan
cpu_ref = cpu_serial_scan(h_input)

# GPU scan
d_input = CuArray(h_input)
gpu_scan_static!(d_input)
gpu_result = collect(d_input)

println("First 10 CPU: ", cpu_ref[1:10])
println("First 10 GPU: ", gpu_result[1:10])
println("Correct? ", cpu_ref ≈ gpu_result)

println("\n=== Timing ===")

println("CPU:")
@btime cpu_serial_scan($h_input);
println("GPU:")
@btime begin
    copyto!(d_input, h_input)   # reset input on GPU
    gpu_scan_static!(d_input)
    synchronize()
end;

First 10 CPU: Float32[0.0, 0.31806326, 0.9628847, 1.1387913, 1.6424044, 2.3615456, 2.6505861, 2.9473846, 3.331805, 3.7266717]
First 10 GPU: Float32[0.0, 0.31806326, 0.9628847, 1.1387913, 1.6424046, 2.3615458, 2.6505864, 2.9473848, 3.3318052, 3.726672]
Correct? true

=== Timing ===
CPU:
  643.114 ns (3 allocations: 4.09 KiB)
GPU:
  26.800 μs (7 allocations: 144 bytes)


## Multi block simplified version

Utilizes the single block version above, so this is also exclusive scan.

In [5]:
using CUDA

# -------------------------
# 1. Phase 1 kernel: scan each block
# -------------------------
function kernel_phase1!(data, block_sums, n)
    T = eltype(data)
    sdata = @cuStaticSharedMem(T, 1024)

    tid = threadIdx().x
    t   = tid - 1

    block_size = 1024
    block = blockIdx().x

    gid = (block - 1) * block_size + tid

    # load
    if gid <= n
        sdata[tid] = data[gid]
    else
        sdata[tid] = zero(T)
    end
    sync_threads()

    # upsweep
    offset = 1
    while offset < block_size
        paircount = block_size ÷ (2 * offset)
        if t < paircount
            ai = offset * (2t + 1)
            bi = offset * (2t + 2)
            sdata[bi] += sdata[ai]
        end
        offset *= 2
        sync_threads()
    end

    # write block sum
    if tid == 1
        block_sums[block] = sdata[block_size]
        sdata[block_size] = zero(T)
    end
    sync_threads()

    # downsweep
    offset = block_size ÷ 2
    while offset ≥ 1
        paircount = block_size ÷ (2 * offset)
        if t < paircount
            ai = offset * (2t + 1)
            bi = offset * (2t + 2)
            tmp = sdata[ai]
            sdata[ai] = sdata[bi]
            sdata[bi] += tmp
        end
        offset ÷= 2
        sync_threads()
    end

    # write back
    if gid <= n
        data[gid] = sdata[tid]
    end

    return nothing
end

# -------------------------
# 2. Phase 3 kernel: add scanned block sums to each block
# -------------------------
function kernel_add_offsets!(data, block_offsets, n)
    block_size = 1024
    block = blockIdx().x
    tid   = threadIdx().x

    gid = (block - 1) * block_size + tid

    if block > 1 && gid <= n
        data[gid] += block_offsets[block]
    end

    return nothing
end

# -------------------------
# 3. Multi-block scan wrapper
# -------------------------
function gpu_scan_multiblock!(x::CuArray)
    N = length(x)
    block_size = 1024
    n_blocks = cld(N, block_size)

    d_block_sums = CuArray(zeros(eltype(x), n_blocks))

    # phase 1
    @cuda threads=block_size blocks=n_blocks kernel_phase1!(x, d_block_sums, N)
    synchronize()

    # phase 2: scan block sums (reuse the above single-block scan)
    if n_blocks > 1
        gpu_scan_static!(d_block_sums)
        synchronize()
    end

    # phase 3
    if n_blocks > 1
        @cuda threads=block_size blocks=n_blocks kernel_add_offsets!(x, d_block_sums, N)
        synchronize()
    end

    return nothing
end

gpu_scan_multiblock! (generic function with 1 method)

### Benchmarking using a large enough problem

Now we can benchmark this paricular GPU version to the serial CPU version.
But don't expect this GPU version to be faster than the CPU version, as it is not using warp optimization (more synchronization overheads), has kernel launching overhead, and suffer from GPU ALU under-utilization (memory bandwidth, prefix sum is memory-bound). Also CPU version has L2/L3 cache benefits.

In [6]:
using BenchmarkTools

# Problem size (large-ish)
N = 2^16
h = rand(Float32, N)
d = CuArray(h)

# Warm-up (important: triggers compilation)
gpu_scan_multiblock!(d)
synchronize()

println("Correctness check:")
cpu = cpu_serial_scan(h)
gpu = collect(d)
println(cpu ≈ gpu)

println("\n=== Benchmark ===")

println("CPU serial exclusive scan:")
@btime cpu_serial_scan($h);

println("\nGPU multi-block exclusive scan:")
@btime begin
    copyto!($d, $h)           # reset GPU input
    gpu_scan_multiblock!($d)
    synchronize()             # wait for GPU completion
end;

Correctness check:
true

=== Benchmark ===
CPU serial exclusive scan:
  39.100 μs (3 allocations: 256.07 KiB)

GPU multi-block exclusive scan:
  127.400 μs (166 allocations: 3.02 KiB)


### Benchmarking using a larger problem

Now try a larger problem size. It helps bring more potential out of this GPU version, but not too much. We can still see the GPU version runs faster though.

Note: the single block version GPU blelloch scan implementation has a baked-in limitation of only assuming 1024 threads per block, so the largest problem size the multi-block version can handle is $1024*1024 = \mathbf{2}^{\mathbf{20}}$ (n_blocks <= 1024). But this is sufficient for demonstrating simple parallelism on GPU.

In [7]:
using BenchmarkTools

# Problem size (larger)
N = 2^20
h = rand(Float32, N)
d = CuArray(h)

# Warm-up (important: triggers compilation)
gpu_scan_multiblock!(d)
synchronize()

println("Correctness check:")
cpu = cpu_serial_scan(h)
gpu = collect(d)
println(cpu ≈ gpu)

println("\n=== Benchmark ===")

println("CPU serial exclusive scan:")
@btime cpu_serial_scan($h);

println("\nGPU multi-block exclusive scan:")
@btime begin
    copyto!($d, $h)           # reset GPU input
    gpu_scan_multiblock!($d)
    synchronize()             # wait for GPU completion
end;

Correctness check:
true

=== Benchmark ===
CPU serial exclusive scan:
  730.100 μs (3 allocations: 4.00 MiB)

GPU multi-block exclusive scan:
  441.700 μs (337 allocations: 9.48 KiB)
